In [1]:
import statsmodels.tools.eval_measures as sm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import os

#### 1. Social housing in Dublin as Percentage of total housing

In [98]:
df = pd.read_csv('./social_housing_number_properties_2015-2019.csv')
df

,STATISTIC Label,Year,Local Electoral Area,UNIT,VALUE
0,Number of HAP Properties,2015,"Borris-In-Ossory-Mountmellick, Laois",Number,0.0
1,Number of HAP Properties,2015,"Portlaoise, Laois",Number,0.0
2,Number of HAP Properties,2015,"Graiguecullen -Portarlington, Laois",Number,NaN
3,Number of HAP Properties,2015,"Carlow, Carlow",Number,48.0
4,Number of HAP Properties,2015,"Tullow, Carlow",Number,18.0
...,...,...,...,...,...
5307,HAP Properties as Percentage of RTB Properties...,2022,"Newbridge, Kildare",%,NaN
5308,HAP Properties as Percentage of RTB Properties...,2022,"Castlecomer, Kilkenny",%,NaN
5309,HAP Properties as Percentage of RTB Properties...,2022,"Kilkenny, Kilkenny",%,NaN
5310,HAP Properties as Percentage of RTB Properties...,2022,"Piltown, Kilkenny",%,NaN


In [99]:
df.dropna(inplace=True)

In [100]:
df

,STATISTIC Label,Year,Local Electoral Area,UNIT,VALUE
0,Number of HAP Properties,2015,"Borris-In-Ossory-Mountmellick, Laois",Number,0.0
1,Number of HAP Properties,2015,"Portlaoise, Laois",Number,0.0
3,Number of HAP Properties,2015,"Carlow, Carlow",Number,48.0
4,Number of HAP Properties,2015,"Tullow, Carlow",Number,18.0
5,Number of HAP Properties,2015,"Muinebeag, Carlow",Number,13.0
...,...,...,...,...,...
5141,HAP Properties as Percentage of RTB Properties...,2021,"Newbridge, Kildare",%,22.3
5142,HAP Properties as Percentage of RTB Properties...,2021,"Castlecomer, Kilkenny",%,28.2
5143,HAP Properties as Percentage of RTB Properties...,2021,"Kilkenny, Kilkenny",%,13.8
5144,HAP Properties as Percentage of RTB Properties...,2021,"Piltown, Kilkenny",%,19.3


In [101]:
df.columns = df.columns.str.lower()

In [102]:
df_dublin = df[df['local electoral area'].str.contains('Dublin')]
df_dublin

,statistic label,year,local electoral area,unit,value
91,Number of HAP Properties,2015,"Lucan, South Dublin",Number,41.0
92,Number of HAP Properties,2015,"Tallaght Central, South Dublin",Number,137.0
93,Number of HAP Properties,2015,"Rathfarnham-Templeogue, South Dublin",Number,7.0
94,Number of HAP Properties,2015,"Firhouse-Bohernabreena, South Dublin",Number,36.0
95,Number of HAP Properties,2015,"Tallaght South, South Dublin",Number,123.0
...,...,...,...,...,...
5086,HAP Properties as Percentage of RTB Properties...,2021,"North Inner City, Dublin City",%,14.2
5087,HAP Properties as Percentage of RTB Properties...,2021,"Clontarf, Dublin City",%,14.0
5088,HAP Properties as Percentage of RTB Properties...,2021,"Donaghmede, Dublin City",%,23.5
5089,HAP Properties as Percentage of RTB Properties...,2021,"Artane-Whitehall, Dublin City",%,26.5


In [105]:
df_dublin.to_csv('./social_housing_dublin_area.csv')

In [106]:
df_pivot = df_dublin.pivot_table(index='year', columns='statistic label', values='value', aggfunc='mean')

In [107]:
df_pivot.reset_index(inplace=True)

In [108]:
df_pivot = df_pivot.rename_axis(None, axis=1)

In [110]:
df_pivot.rename(columns={'hap properties as percentage of eircodes in area':'as perc. of all postcodes',
                         'hap properties as percentage of rtb properties in area':'as perc. of rental properties',
                         'number of hap properties':'total assisted housing units',
                         'percentage change in number of hap properties from previous year':'perc. change in total from previous year'}, inplace=True)

In [111]:
cols = ['year', 
        'total assisted housing units', 
        'as perc. of rental properties', 
        'as perc. of all postcodes', 
        'perc. change in total from previous year']

df_pivot = df_pivot[cols]

In [277]:
df_pivot#.to_csv('./social_housing_dublin_2015_2022.csv')

,year,total assisted housing units,as perc. of rental properties,as perc. of all postcodes,perc. change in total from previous year
0,2015,56.100000,NaN,0.227778,NaN
1,2016,108.555556,NaN,0.738889,333.777778
2,2017,251.388889,10.238889,1.566667,299.522222
3,2018,383.666667,14.927778,2.266667,65.955556
4,2019,503.333333,18.544444,2.855556,35.766667
5,2020,642.833333,25.061111,3.488889,33.366667
6,2021,755.722222,22.905556,4.000000,19.183333
7,2022,749.777778,NaN,3.916667,-0.844444


#### 2. Profile of people receiving housing assistance

In [251]:
df2 = pd.read_csv('./hap_demographic_2015_2019.csv')
df2

,Statistic Label,Year,Local Authority,Family Type,UNIT,VALUE
0,Number of Households Exiting HAP,2014,All Counties,All Family Types,Number,0.0
1,Number of Households Exiting HAP,2014,All Counties,Couple,Number,0.0
2,Number of Households Exiting HAP,2014,All Counties,Couple 1 Child,Number,0.0
3,Number of Households Exiting HAP,2014,All Counties,Couple 2 Children,Number,0.0
4,Number of Households Exiting HAP,2014,All Counties,Couple 3+ Children,Number,0.0
...,...,...,...,...,...,...
5755,Number of Households in HAP,2019,Monaghan,Single,Number,162.0
5756,Number of Households in HAP,2019,Monaghan,Single 1 Child,Number,141.0
5757,Number of Households in HAP,2019,Monaghan,Single 2 Children,Number,95.0
5758,Number of Households in HAP,2019,Monaghan,Single 3+ Children,Number,52.0


In [252]:
df2.columns = df2.columns.str.lower()
df2

,statistic label,year,local authority,family type,unit,value
0,Number of Households Exiting HAP,2014,All Counties,All Family Types,Number,0.0
1,Number of Households Exiting HAP,2014,All Counties,Couple,Number,0.0
2,Number of Households Exiting HAP,2014,All Counties,Couple 1 Child,Number,0.0
3,Number of Households Exiting HAP,2014,All Counties,Couple 2 Children,Number,0.0
4,Number of Households Exiting HAP,2014,All Counties,Couple 3+ Children,Number,0.0
...,...,...,...,...,...,...
5755,Number of Households in HAP,2019,Monaghan,Single,Number,162.0
5756,Number of Households in HAP,2019,Monaghan,Single 1 Child,Number,141.0
5757,Number of Households in HAP,2019,Monaghan,Single 2 Children,Number,95.0
5758,Number of Households in HAP,2019,Monaghan,Single 3+ Children,Number,52.0


In [119]:
df2['local authority'].unique()

array(['All Counties', 'Carlow', 'Dublin City', 'South Dublin', 'Fingal',
       'Dún Laoghaire-Rathdown', 'Kildare', 'Kilkenny', 'Laois',
       'Longford', 'Louth', 'Meath', 'Offaly', 'Westmeath', 'Wexford',
       'Wicklow', 'Clare', 'Cork City', 'Cork County', 'Kerry',
       'Limerick', 'Tipperary', 'Waterford', 'Galway City',
       'Galway County', 'Leitrim', 'Mayo', 'Roscommon', 'Sligo', 'Cavan',
       'Donegal', 'Monaghan'], dtype=object)

In [120]:
df2_dublin = df2[(df2['local authority'] == 'Dublin City')|(df2['local authority'] == 'South Dublin')|(df2['local authority'] =='Fingal')|
       (df2['local authority'] =='Dún Laoghaire-Rathdown')]
df2_dublin

,statistic label,year,local authority,family type,unit,value
20,Number of Households Exiting HAP,2014,Dublin City,All Family Types,Number,0.0
21,Number of Households Exiting HAP,2014,Dublin City,Couple,Number,0.0
22,Number of Households Exiting HAP,2014,Dublin City,Couple 1 Child,Number,0.0
23,Number of Households Exiting HAP,2014,Dublin City,Couple 2 Children,Number,0.0
24,Number of Households Exiting HAP,2014,Dublin City,Couple 3+ Children,Number,0.0
...,...,...,...,...,...,...
5495,Number of Households in HAP,2019,Dún Laoghaire-Rathdown,Single,Number,104.0
5496,Number of Households in HAP,2019,Dún Laoghaire-Rathdown,Single 1 Child,Number,290.0
5497,Number of Households in HAP,2019,Dún Laoghaire-Rathdown,Single 2 Children,Number,151.0
5498,Number of Households in HAP,2019,Dún Laoghaire-Rathdown,Single 3+ Children,Number,41.0


In [139]:
df2_dublin['statistic label'].unique()

array(['Number of Households Exiting HAP',
       'Number of Households Starting in HAP',
       'Number of Households in HAP'], dtype=object)

In [124]:
hap_exits = df2_dublin[df2_dublin['statistic label'] == 'Number of Households Exiting HAP']
hap_exits

,statistic label,year,local authority,family type,unit,value
20,Number of Households Exiting HAP,2014,Dublin City,All Family Types,Number,0.0
21,Number of Households Exiting HAP,2014,Dublin City,Couple,Number,0.0
22,Number of Households Exiting HAP,2014,Dublin City,Couple 1 Child,Number,0.0
23,Number of Households Exiting HAP,2014,Dublin City,Couple 2 Children,Number,0.0
24,Number of Households Exiting HAP,2014,Dublin City,Couple 3+ Children,Number,0.0
...,...,...,...,...,...,...
1655,Number of Households Exiting HAP,2019,Dún Laoghaire-Rathdown,Single,Number,12.0
1656,Number of Households Exiting HAP,2019,Dún Laoghaire-Rathdown,Single 1 Child,Number,26.0
1657,Number of Households Exiting HAP,2019,Dún Laoghaire-Rathdown,Single 2 Children,Number,13.0
1658,Number of Households Exiting HAP,2019,Dún Laoghaire-Rathdown,Single 3+ Children,Number,NaN


In [140]:
hap_starts = df2_dublin[df2_dublin['statistic label'] == 'Number of Households Starting in HAP']
hap_starts

,statistic label,year,local authority,family type,unit,value
1940,Number of Households Starting in HAP,2014,Dublin City,All Family Types,Number,0.0
1941,Number of Households Starting in HAP,2014,Dublin City,Couple,Number,0.0
1942,Number of Households Starting in HAP,2014,Dublin City,Couple 1 Child,Number,0.0
1943,Number of Households Starting in HAP,2014,Dublin City,Couple 2 Children,Number,0.0
1944,Number of Households Starting in HAP,2014,Dublin City,Couple 3+ Children,Number,0.0
...,...,...,...,...,...,...
3575,Number of Households Starting in HAP,2019,Dún Laoghaire-Rathdown,Single,Number,37.0
3576,Number of Households Starting in HAP,2019,Dún Laoghaire-Rathdown,Single 1 Child,Number,96.0
3577,Number of Households Starting in HAP,2019,Dún Laoghaire-Rathdown,Single 2 Children,Number,48.0
3578,Number of Households Starting in HAP,2019,Dún Laoghaire-Rathdown,Single 3+ Children,Number,5.0


In [141]:
in_hap = df2_dublin[df2_dublin['statistic label'] == 'Number of Households in HAP']
in_hap

,statistic label,year,local authority,family type,unit,value
3860,Number of Households in HAP,2014,Dublin City,All Family Types,Number,0.0
3861,Number of Households in HAP,2014,Dublin City,Couple,Number,0.0
3862,Number of Households in HAP,2014,Dublin City,Couple 1 Child,Number,0.0
3863,Number of Households in HAP,2014,Dublin City,Couple 2 Children,Number,0.0
3864,Number of Households in HAP,2014,Dublin City,Couple 3+ Children,Number,0.0
...,...,...,...,...,...,...
5495,Number of Households in HAP,2019,Dún Laoghaire-Rathdown,Single,Number,104.0
5496,Number of Households in HAP,2019,Dún Laoghaire-Rathdown,Single 1 Child,Number,290.0
5497,Number of Households in HAP,2019,Dún Laoghaire-Rathdown,Single 2 Children,Number,151.0
5498,Number of Households in HAP,2019,Dún Laoghaire-Rathdown,Single 3+ Children,Number,41.0


In [126]:
hap_exits.rename(columns={'value':'number of households exiting hap'}, inplace=True)

/var/folders/fq/w4n1nx9n5flfw6tv_nzn8kyr0000gn/T/ipykernel_30243/2778313849.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hap_exits.rename(columns={'value':'number of households exiting hap'}, inplace=True)


In [127]:
hap_exits.pop('unit')

20      Number
21      Number
22      Number
23      Number
24      Number
         ...  
1655    Number
1656    Number
1657    Number
1658    Number
1659    Number
Name: unit, Length: 240, dtype: object

In [128]:
hap_exits.pop('statistic label')

20      Number of Households Exiting HAP
21      Number of Households Exiting HAP
22      Number of Households Exiting HAP
23      Number of Households Exiting HAP
24      Number of Households Exiting HAP
                      ...               
1655    Number of Households Exiting HAP
1656    Number of Households Exiting HAP
1657    Number of Households Exiting HAP
1658    Number of Households Exiting HAP
1659    Number of Households Exiting HAP
Name: statistic label, Length: 240, dtype: object

In [138]:
hap_exits

,year,local authority,family type,number of households exiting hap
20,2014,Dublin City,All Family Types,0.0
21,2014,Dublin City,Couple,0.0
22,2014,Dublin City,Couple 1 Child,0.0
23,2014,Dublin City,Couple 2 Children,0.0
24,2014,Dublin City,Couple 3+ Children,0.0
...,...,...,...,...
1655,2019,Dún Laoghaire-Rathdown,Single,12.0
1656,2019,Dún Laoghaire-Rathdown,Single 1 Child,26.0
1657,2019,Dún Laoghaire-Rathdown,Single 2 Children,13.0
1658,2019,Dún Laoghaire-Rathdown,Single 3+ Children,NaN


In [142]:
hap_starts.rename(columns={'value':'number of households starting hap'}, inplace=True)

/var/folders/fq/w4n1nx9n5flfw6tv_nzn8kyr0000gn/T/ipykernel_30243/3714767988.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hap_starts.rename(columns={'value':'number of households starting hap'}, inplace=True)


In [143]:
hap_starts.pop('unit')

1940    Number
1941    Number
1942    Number
1943    Number
1944    Number
         ...  
3575    Number
3576    Number
3577    Number
3578    Number
3579    Number
Name: unit, Length: 240, dtype: object

In [144]:
hap_starts.pop('statistic label')

1940    Number of Households Starting in HAP
1941    Number of Households Starting in HAP
1942    Number of Households Starting in HAP
1943    Number of Households Starting in HAP
1944    Number of Households Starting in HAP
                        ...                 
3575    Number of Households Starting in HAP
3576    Number of Households Starting in HAP
3577    Number of Households Starting in HAP
3578    Number of Households Starting in HAP
3579    Number of Households Starting in HAP
Name: statistic label, Length: 240, dtype: object

In [145]:
hap_starts

,year,local authority,family type,number of households starting hap
1940,2014,Dublin City,All Family Types,0.0
1941,2014,Dublin City,Couple,0.0
1942,2014,Dublin City,Couple 1 Child,0.0
1943,2014,Dublin City,Couple 2 Children,0.0
1944,2014,Dublin City,Couple 3+ Children,0.0
...,...,...,...,...
3575,2019,Dún Laoghaire-Rathdown,Single,37.0
3576,2019,Dún Laoghaire-Rathdown,Single 1 Child,96.0
3577,2019,Dún Laoghaire-Rathdown,Single 2 Children,48.0
3578,2019,Dún Laoghaire-Rathdown,Single 3+ Children,5.0


In [146]:
in_hap.rename(columns={'value':'number of households in hap'}, inplace=True)

/var/folders/fq/w4n1nx9n5flfw6tv_nzn8kyr0000gn/T/ipykernel_30243/3548965163.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  in_hap.rename(columns={'value':'number of households in hap'}, inplace=True)


In [147]:
in_hap.pop('unit')

3860    Number
3861    Number
3862    Number
3863    Number
3864    Number
         ...  
5495    Number
5496    Number
5497    Number
5498    Number
5499    Number
Name: unit, Length: 240, dtype: object

In [148]:
in_hap.pop('statistic label')

3860    Number of Households in HAP
3861    Number of Households in HAP
3862    Number of Households in HAP
3863    Number of Households in HAP
3864    Number of Households in HAP
                   ...             
5495    Number of Households in HAP
5496    Number of Households in HAP
5497    Number of Households in HAP
5498    Number of Households in HAP
5499    Number of Households in HAP
Name: statistic label, Length: 240, dtype: object

In [149]:
in_hap

,year,local authority,family type,number of households in hap
3860,2014,Dublin City,All Family Types,0.0
3861,2014,Dublin City,Couple,0.0
3862,2014,Dublin City,Couple 1 Child,0.0
3863,2014,Dublin City,Couple 2 Children,0.0
3864,2014,Dublin City,Couple 3+ Children,0.0
...,...,...,...,...
5495,2019,Dún Laoghaire-Rathdown,Single,104.0
5496,2019,Dún Laoghaire-Rathdown,Single 1 Child,290.0
5497,2019,Dún Laoghaire-Rathdown,Single 2 Children,151.0
5498,2019,Dún Laoghaire-Rathdown,Single 3+ Children,41.0


In [150]:
df1 = in_hap
df2 = hap_starts

df3 = df1.merge(df2, how='outer', on=['year', 'local authority', 'family type'])

In [152]:
df1 = df3
df2 = hap_exits

hap_full = df1.merge(df2, how='outer', on=['year', 'local authority', 'family type'])

In [344]:
hap_full = hap_full[hap_full['year'] >2014]
hap_full

,year,local authority,family type,number of households in hap,number of households starting hap,number of households exiting hap
40,2015,Dublin City,All Family Types,50.0,50.0,0.0
41,2015,Dublin City,Couple,NaN,NaN,0.0
42,2015,Dublin City,Couple 1 Child,NaN,NaN,NaN
43,2015,Dublin City,Couple 2 Children,NaN,NaN,0.0
44,2015,Dublin City,Couple 3+ Children,NaN,NaN,0.0
...,...,...,...,...,...,...
235,2019,Dún Laoghaire-Rathdown,Single,104.0,37.0,12.0
236,2019,Dún Laoghaire-Rathdown,Single 1 Child,290.0,96.0,26.0
237,2019,Dún Laoghaire-Rathdown,Single 2 Children,151.0,48.0,13.0
238,2019,Dún Laoghaire-Rathdown,Single 3+ Children,41.0,5.0,NaN


In [345]:
hap_full_grouped = hap_full.groupby(['year'])[['number of households in hap', 'number of households starting hap', 'number of households exiting hap']].sum()
hap_full_grouped

,number of households in hap,number of households starting hap,number of households exiting hap
year,,,
2015,1214.0,1196.0,0.0
2016,4848.0,2881.0,107.0
2017,12963.0,7856.0,330.0
2018,22573.0,9753.0,1133.0
2019,29921.0,9017.0,2346.0


In [349]:
hap_full_grouped.to_csv('hap_demographic_dublin.csv')

In [339]:
hap_family = hap_full.groupby(['year', 'family type'])[['number of households in hap', 'number of households starting hap', 'number of households exiting hap']].sum()

In [341]:
hap_family.reset_index(inplace=True)

In [342]:
hap_family = hap_family[hap_family['year'] >2014]
hap_family

,year,family type,number of households in hap,number of households starting hap,number of households exiting hap
10,2015,All Family Types,620.0,610.0,0.0
11,2015,Couple,9.0,9.0,0.0
12,2015,Couple 1 Child,47.0,46.0,0.0
13,2015,Couple 2 Children,77.0,77.0,0.0
14,2015,Couple 3+ Children,72.0,71.0,0.0
15,2015,Other Family Structures,29.0,29.0,0.0
16,2015,Single,34.0,32.0,0.0
17,2015,Single 1 Child,171.0,167.0,0.0
18,2015,Single 2 Children,95.0,95.0,0.0
19,2015,Single 3+ Children,60.0,60.0,0.0


In [343]:
hap_family.to_csv('./hap_family.csv')

#### 3. Percentage of people exiting housing assistance and reasons for exit (state-wide)

In [155]:
exit_perc = pd.read_csv('./hap_exits_2015-2019.csv')

In [195]:
exit_perc.columns = exit_perc.columns.str.lower()
exit_perc

,statistic,year,unit,accommodation/landlord non compliance,deceased tenant,landlord ends tenancy,miscellaneous,social housing,tenant ends tenancy,tenant non-compliance
0,Percentage of Exiting HAP tenants,2015,%,1.333333,2.222222,12.888889,11.111111,17.777778,49.333333,5.333333
1,Percentage of Exiting HAP tenants,2016,%,0.583658,3.015564,14.785992,10.603113,20.038911,36.186770,14.785992
2,Percentage of Exiting HAP tenants,2017,%,0.311943,3.342246,15.329768,10.115865,28.297683,32.308378,10.294118
3,Percentage of Exiting HAP tenants,2018,%,0.215827,2.494005,17.098321,8.896882,32.517986,30.791367,7.985612
4,Percentage of Exiting HAP tenants,2019,%,0.356506,1.886512,16.027926,10.932858,34.982175,27.287582,8.526441


In [253]:
hap_exits_state = df2[df2['statistic label'] == 'Number of Households Exiting HAP']
hap_exits_state

,statistic label,year,local authority,family type,unit,value
0,Number of Households Exiting HAP,2014,All Counties,All Family Types,Number,0.0
1,Number of Households Exiting HAP,2014,All Counties,Couple,Number,0.0
2,Number of Households Exiting HAP,2014,All Counties,Couple 1 Child,Number,0.0
3,Number of Households Exiting HAP,2014,All Counties,Couple 2 Children,Number,0.0
4,Number of Households Exiting HAP,2014,All Counties,Couple 3+ Children,Number,0.0
...,...,...,...,...,...,...
1915,Number of Households Exiting HAP,2019,Monaghan,Single,Number,24.0
1916,Number of Households Exiting HAP,2019,Monaghan,Single 1 Child,Number,21.0
1917,Number of Households Exiting HAP,2019,Monaghan,Single 2 Children,Number,22.0
1918,Number of Households Exiting HAP,2019,Monaghan,Single 3+ Children,Number,12.0


In [254]:
hap_exits_state.rename(columns={'value':'number of households exiting hap'}, inplace=True)

/var/folders/fq/w4n1nx9n5flfw6tv_nzn8kyr0000gn/T/ipykernel_30243/1226746790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hap_exits_state.rename(columns={'value':'number of households exiting hap'}, inplace=True)


In [255]:
hap_exits_state.pop('statistic label')

0       Number of Households Exiting HAP
1       Number of Households Exiting HAP
2       Number of Households Exiting HAP
3       Number of Households Exiting HAP
4       Number of Households Exiting HAP
                      ...               
1915    Number of Households Exiting HAP
1916    Number of Households Exiting HAP
1917    Number of Households Exiting HAP
1918    Number of Households Exiting HAP
1919    Number of Households Exiting HAP
Name: statistic label, Length: 1920, dtype: object

In [256]:
hap_exits_state.pop('unit')

0       Number
1       Number
2       Number
3       Number
4       Number
         ...  
1915    Number
1916    Number
1917    Number
1918    Number
1919    Number
Name: unit, Length: 1920, dtype: object

In [257]:
hap_exits_grouped = hap_exits_state.groupby(['year'])[['number of households exiting hap']].sum()

In [258]:
hap_exits_grouped.reset_index(inplace=True)
hap_exits_grouped

,year,number of households exiting hap
0,2014,0.0
1,2015,823.0
2,2016,3941.0
3,2017,8813.0
4,2018,16530.0
5,2019,26866.0


In [260]:
df1 = exit_perc
df2 = hap_exits_grouped

hap_exits_perc = df1.merge(df2, how='outer', on='year')

In [262]:
hap_exits_perc.pop('statistic')

0    Percentage of Exiting HAP tenants
1    Percentage of Exiting HAP tenants
2    Percentage of Exiting HAP tenants
3    Percentage of Exiting HAP tenants
4    Percentage of Exiting HAP tenants
5                                  NaN
Name: statistic, dtype: object

In [330]:
hap_exits_perc.pop('unit')

0      %
1      %
2      %
3      %
4      %
5    NaN
Name: unit, dtype: object

In [331]:
cols = ['year', 
        'number of households exiting hap', 
        'tenant ends tenancy', 
        'landlord ends tenancy', 
        'social housing',
        'accommodation/landlord non compliance',
        'tenant non-compliance',
        'deceased tenant',
        'miscellaneous'
        ]

In [334]:
hap_exits_perc = hap_exits_perc[cols]

,year,number of households exiting hap,tenant ends tenancy,landlord ends tenancy,social housing,accommodation/landlord non compliance,tenant non-compliance,deceased tenant,miscellaneous
0,2015,823.0,49.333333,12.888889,17.777778,1.333333,5.333333,2.222222,11.111111
1,2016,3941.0,36.186770,14.785992,20.038911,0.583658,14.785992,3.015564,10.603113
2,2017,8813.0,32.308378,15.329768,28.297683,0.311943,10.294118,3.342246,10.115865
3,2018,16530.0,30.791367,17.098321,32.517986,0.215827,7.985612,2.494005,8.896882
4,2019,26866.0,27.287582,16.027926,34.982175,0.356506,8.526441,1.886512,10.932858
5,2014,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [347]:
hap_exits_perc = hap_exits_perc[hap_exits_perc['year'] >2014]
hap_exits_perc

,year,number of households exiting hap,tenant ends tenancy,landlord ends tenancy,social housing,accommodation/landlord non compliance,tenant non-compliance,deceased tenant,miscellaneous
0,2015,823.0,49.333333,12.888889,17.777778,1.333333,5.333333,2.222222,11.111111
1,2016,3941.0,36.186770,14.785992,20.038911,0.583658,14.785992,3.015564,10.603113
2,2017,8813.0,32.308378,15.329768,28.297683,0.311943,10.294118,3.342246,10.115865
3,2018,16530.0,30.791367,17.098321,32.517986,0.215827,7.985612,2.494005,8.896882
4,2019,26866.0,27.287582,16.027926,34.982175,0.356506,8.526441,1.886512,10.932858


In [348]:
hap_exits_perc.to_csv('./hap_exits_percentage.csv')

#### 4. Homeless people given houses through hap scheme

In [283]:
hap_homeless = pd.read_csv('./homeless_given_houses_2015-2019.csv')
hap_homeless

,Statistic Label,Year,Local Authority,UNIT,VALUE
0,Number of HAP Homeless referred from Homeless ...,2015,All Counties,Number,310.00
1,Number of HAP Homeless referred from Homeless ...,2015,Carlow,Number,NaN
2,Number of HAP Homeless referred from Homeless ...,2015,Dublin City,Number,48.00
3,Number of HAP Homeless referred from Homeless ...,2015,South Dublin,Number,94.00
4,Number of HAP Homeless referred from Homeless ...,2015,Fingal,Number,7.00
...,...,...,...,...,...
315,Percentage of HAP Homeless referred from Homel...,2019,Roscommon,%,6.21
316,Percentage of HAP Homeless referred from Homel...,2019,Sligo,%,9.71
317,Percentage of HAP Homeless referred from Homel...,2019,Cavan,%,6.54
318,Percentage of HAP Homeless referred from Homel...,2019,Donegal,%,2.39


In [284]:
hap_homeless.columns = hap_homeless.columns.str.lower()

In [285]:
hap_homeless['statistic label'].unique()

array(['Number of HAP Homeless referred from Homeless services',
       'Percentage of HAP Homeless referred from Homeless services'],
      dtype=object)

In [286]:
hap_homeless_1 = hap_homeless[hap_homeless['statistic label'] == 'Percentage of HAP Homeless referred from Homeless services']

In [ ]:
hap_homeless_1.pop('statistic label')

In [288]:
hap_homeless_1.pop('unit')

160    %
161    %
162    %
163    %
164    %
      ..
315    %
316    %
317    %
318    %
319    %
Name: unit, Length: 160, dtype: object

In [289]:
hap_homeless_1.rename(columns={'value':'percentage of total hap referred from homeless services'}, inplace=True)
hap_homeless_1

/var/folders/fq/w4n1nx9n5flfw6tv_nzn8kyr0000gn/T/ipykernel_30243/3302939154.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hap_homeless_1.rename(columns={'value':'percentage of total hap referred from homeless services'}, inplace=True)


,year,local authority,percentage of total hap referred from homeless services
160,2015,All Counties,5.79
161,2015,Carlow,NaN
162,2015,Dublin City,97.96
163,2015,South Dublin,16.91
164,2015,Fingal,87.50
...,...,...,...
315,2019,Roscommon,6.21
316,2019,Sligo,9.71
317,2019,Cavan,6.54
318,2019,Donegal,2.39


In [290]:
hap_homeless_1['local authority'].unique()

array(['All Counties', 'Carlow', 'Dublin City', 'South Dublin', 'Fingal',
       'Dún Laoghaire-Rathdown', 'Kildare', 'Kilkenny', 'Laois',
       'Longford', 'Louth', 'Meath', 'Offaly', 'Westmeath', 'Wexford',
       'Wicklow', 'Clare', 'Cork City', 'Cork County', 'Kerry',
       'Limerick', 'Tipperary', 'Waterford', 'Galway City',
       'Galway County', 'Leitrim', 'Mayo', 'Roscommon', 'Sligo', 'Cavan',
       'Donegal', 'Monaghan'], dtype=object)

In [292]:
hap_homeless_1.drop(hap_homeless_1[hap_homeless_1['local authority'] == 'All Counties'].index, inplace = True)
hap_homeless_1

/var/folders/fq/w4n1nx9n5flfw6tv_nzn8kyr0000gn/T/ipykernel_30243/2942641819.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hap_homeless_1.drop(hap_homeless_1[hap_homeless_1['local authority'] == 'All Counties'].index, inplace = True)


,year,local authority,percentage of total hap referred from homeless services
161,2015,Carlow,NaN
162,2015,Dublin City,97.96
163,2015,South Dublin,16.91
164,2015,Fingal,87.50
165,2015,Dún Laoghaire-Rathdown,NaN
...,...,...,...
315,2019,Roscommon,6.21
316,2019,Sligo,9.71
317,2019,Cavan,6.54
318,2019,Donegal,2.39


In [237]:
hh1_grouped = hap_homeless_1.groupby(['year'])[['percentage of total hap referred from homeless services']].mean()
hh1_grouped

,percentage of total hap referred from homeless services
year,
2015,14.175882
2016,10.772083
2017,10.942800
2018,11.189615
2019,15.832333


In [293]:
hap_homeless_2 = hap_homeless[hap_homeless['statistic label'] == 'Number of HAP Homeless referred from Homeless services']

In [294]:
hap_homeless_2.rename(columns={'value':'number of hap referred from homeless services'}, inplace=True)

/var/folders/fq/w4n1nx9n5flfw6tv_nzn8kyr0000gn/T/ipykernel_30243/1080701602.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hap_homeless_2.rename(columns={'value':'number of hap referred from homeless services'}, inplace=True)


In [295]:
hap_homeless_2.pop('statistic label')

0      Number of HAP Homeless referred from Homeless ...
1      Number of HAP Homeless referred from Homeless ...
2      Number of HAP Homeless referred from Homeless ...
3      Number of HAP Homeless referred from Homeless ...
4      Number of HAP Homeless referred from Homeless ...
                             ...                        
155    Number of HAP Homeless referred from Homeless ...
156    Number of HAP Homeless referred from Homeless ...
157    Number of HAP Homeless referred from Homeless ...
158    Number of HAP Homeless referred from Homeless ...
159    Number of HAP Homeless referred from Homeless ...
Name: statistic label, Length: 160, dtype: object

In [296]:
hap_homeless_2.pop('unit')

0      Number
1      Number
2      Number
3      Number
4      Number
        ...  
155    Number
156    Number
157    Number
158    Number
159    Number
Name: unit, Length: 160, dtype: object

In [297]:
hap_homeless_2.drop(hap_homeless_2[hap_homeless_2['local authority'] == 'All Counties'].index, inplace = True)
hap_homeless_2

/var/folders/fq/w4n1nx9n5flfw6tv_nzn8kyr0000gn/T/ipykernel_30243/554305442.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hap_homeless_2.drop(hap_homeless_2[hap_homeless_2['local authority'] == 'All Counties'].index, inplace = True)


,year,local authority,number of hap referred from homeless services
1,2015,Carlow,NaN
2,2015,Dublin City,48.0
3,2015,South Dublin,94.0
4,2015,Fingal,7.0
5,2015,Dún Laoghaire-Rathdown,NaN
...,...,...,...
155,2019,Roscommon,10.0
156,2019,Sligo,20.0
157,2019,Cavan,10.0
158,2019,Donegal,9.0


In [241]:
hh2_grouped = hap_homeless_2.groupby(['year'])[['number of hap referred from homeless services']].sum()
hh2_grouped

,number of hap referred from homeless services
year,
2015,297.0
2016,826.0
2017,2167.0
2018,2617.0
2019,3067.0


In [267]:
hap_homeless_2[hap_homeless_2['year'] == 2019]['number of hap referred from homeless services'].sum()

3067.0

In [275]:
df1 = hh2_grouped
df2 = hh1_grouped

hh_state_grouped = df1.merge(df2, how='outer', on='year')

In [278]:
hh_state_grouped.to_csv('.hap_homeless_statewide.csv')

In [311]:
hap_homeless_1

,year,local authority,percentage of total hap referred from homeless services
161,2015,Carlow,NaN
162,2015,Dublin City,97.96
163,2015,South Dublin,16.91
164,2015,Fingal,87.50
165,2015,Dún Laoghaire-Rathdown,NaN
...,...,...,...
315,2019,Roscommon,6.21
316,2019,Sligo,9.71
317,2019,Cavan,6.54
318,2019,Donegal,2.39


In [315]:
df11 = hap_homeless_2
df12 = hap_homeless_1

hap_homeless = df11.merge(df12, how='outer', on=['year', 'local authority'])

In [316]:
hap_homeless['local authority'].unique()

array(['Carlow', 'Dublin City', 'South Dublin', 'Fingal',
       'Dún Laoghaire-Rathdown', 'Kildare', 'Kilkenny', 'Laois',
       'Longford', 'Louth', 'Meath', 'Offaly', 'Westmeath', 'Wexford',
       'Wicklow', 'Clare', 'Cork City', 'Cork County', 'Kerry',
       'Limerick', 'Tipperary', 'Waterford', 'Galway City',
       'Galway County', 'Leitrim', 'Mayo', 'Roscommon', 'Sligo', 'Cavan',
       'Donegal', 'Monaghan'], dtype=object)

In [326]:
hap_homeless_dublin = hap_homeless[(hap_homeless['local authority'] == 'Dublin City')|(hap_homeless['local authority'] == 'South Dublin')|(hap_homeless['local authority'] =='Fingal')|
       (hap_homeless['local authority'] =='Dún Laoghaire-Rathdown')]
hap_homeless_dublin

,year,local authority,number of hap referred from homeless services,percentage of total hap referred from homeless services
1,2015,Dublin City,48.0,97.96
2,2015,South Dublin,94.0,16.91
3,2015,Fingal,7.0,87.50
4,2015,Dún Laoghaire-Rathdown,NaN,NaN
32,2016,Dublin City,421.0,97.45
33,2016,South Dublin,129.0,13.54
34,2016,Fingal,63.0,96.92
35,2016,Dún Laoghaire-Rathdown,NaN,NaN
63,2017,Dublin City,857.0,52.32
64,2017,South Dublin,342.0,29.11


In [328]:
hap_homeless_grouped = hap_homeless_dublin.groupby('year').agg({'number of hap referred from homeless services':'sum', 
                         'percentage of total hap referred from homeless services':'mean'})
hap_homeless_grouped

,number of hap referred from homeless services,percentage of total hap referred from homeless services
year,,
2015,149.0,67.456667
2016,613.0,69.303333
2017,1671.0,41.895000
2018,1949.0,37.972500
2019,1802.0,41.195000


In [356]:
hap_homeless_grouped.to_csv('./hap_homeless_dublin.csv')